In [139]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import re
import requests
import time

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

In [128]:
ump_scores_df = pd.read_csv('D:\Downloads\games (1).csv')

##drop nulls, shortened season
ump_scores_df = ump_scores_df.dropna(axis='index').sort_values('Date')
ump_scores_df = ump_scores_df.set_axis(['date', 'umpire', 'home', 'away', 'home_score', 'away_score', 'called_pitches', 'called_wrong', 'accuracy', 'consistency', 'favor', 'total_run_impact'], axis=1)
covid_2020 = ump_scores_df.date.str.contains('2020')
ump_scores_df = ump_scores_df[~covid_2020].reset_index(drop=True)

display(ump_scores_df)

,date,umpire,home,away,home_score,away_score,called_pitches,called_wrong,accuracy,consistency,favor,total_run_impact
0,2015-04-05,Mike Winters,CHC,STL,0,3,158.0,17.0,89.2,94.9,1.10,4.36
1,2015-04-06,John Hirschbeck,CIN,PIT,5,2,117.0,18.0,84.6,87.2,-0.45,2.21
2,2015-04-06,Joe West,DET,MIN,4,0,109.0,8.0,92.7,96.3,-0.50,0.84
3,2015-04-06,Gerry Davis,PHI,BOS,0,8,171.0,18.0,89.5,94.7,-1.00,2.84
4,2015-04-06,Ted Barrett,HOU,CLE,2,0,124.0,21.0,83.1,91.9,0.51,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...
13484,2021-10-27,Ron Kulpa,HOU,ATL,7,2,120.0,5.0,95.8,96.7,0.10,0.50
13485,2021-10-29,Tom Hallion,ATL,HOU,2,0,146.0,5.0,96.6,93.8,-0.09,0.41
13486,2021-10-30,Dan Bellino,ATL,HOU,3,2,153.0,14.0,90.8,94.1,-0.26,1.48
13487,2021-10-31,Ted Barrett,ATL,HOU,5,9,171.0,14.0,91.8,88.3,0.61,1.47


In [129]:
##cleaning of weather data
seattle_weather_df = pd.read_csv('D:\Downloads\\weather_seattle_dome.csv')
seattle_weather_df = seattle_weather_df[['DATE', 'HourlyPrecipitation', 'HourlyRelativeHumidity', 'HourlyWetBulbTemperature', 'HourlyWindSpeed']]
seattle_weather_df = seattle_weather_df.dropna(axis='index').reset_index(drop=True)

seattle_weather_df.HourlyPrecipitation = seattle_weather_df.HourlyPrecipitation.replace('T', '0.00').str.replace('s', '')
seattle_weather_df

C:\Users\Alex7\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (35,36,42,43,44,50,52,53,57,72,74,76,77,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,DATE,HourlyPrecipitation,HourlyRelativeHumidity,HourlyWetBulbTemperature,HourlyWindSpeed
0,2015-04-06T00:53:00,0.00,71.0,42.0,10.0
1,2015-04-06T01:53:00,0.00,73.0,42.0,9.0
2,2015-04-06T02:53:00,0.00,77.0,42.0,11.0
3,2015-04-06T03:53:00,0.00,80.0,42.0,8.0
4,2015-04-06T04:37:00,0.00,81.0,42.0,8.0
...,...,...,...,...,...
60984,2021-10-03T19:53:00,0.00,75.0,50.0,7.0
60985,2021-10-03T20:53:00,0.00,80.0,49.0,7.0
60986,2021-10-03T21:53:00,0.00,77.0,49.0,7.0
60987,2021-10-03T22:53:00,0.00,80.0,47.0,6.0


In [130]:
##bringing in scraped data
day_night_df = pd.read_csv('Data\\day_night_df.csv').drop(columns='Unnamed: 0').set_axis(['day_night'], axis=1)
start_time_df = pd.read_csv('Data\\start_time_df.csv').drop(columns='Unnamed: 0').set_axis(['start_time'], axis=1)
duration_df = pd.read_csv('Data\\duration_df.csv').drop(columns='Unnamed: 0').set_axis(['duration'], axis=1)
attendance_df = pd.read_csv('Data\\day_night_df.csv').drop(columns='Unnamed: 0').set_axis(['attendance'], axis=1)

day_night_df.day_night = day_night_df.day_night.str.extract(r'((?<=\d\:\d{2}</div><div>).*?(?=</div><div>))')
start_time_df.start_time = start_time_df.start_time.str.extract(r'((?<=Time: ).*?(?= Local))')
duration_df.duration = duration_df.duration.str.extract(r'((?<=n</strong>: ).*?(?=</div>))')
attendance_df.attendance = attendance_df.attendance.str.extract(r'((?<=ce</strong>: ).*?(?=</div>))')

In [131]:
ump_scores_df['day_night'] = day_night_df.day_night
ump_scores_df['start_time'] = start_time_df.start_time
ump_scores_df['duration'] = duration_df.duration
ump_scores_df['attendance'] = attendance_df.attendance
ump_scores_df

,date,umpire,home,away,home_score,away_score,called_pitches,called_wrong,accuracy,consistency,favor,total_run_impact,day_night,start_time,duration,attendance
0,2015-04-05,Mike Winters,CHC,STL,0,3,158.0,17.0,89.2,94.9,1.10,4.36,"Night Game, on grass",7:17 p.m.,3:04,"35,055"
1,2015-04-06,John Hirschbeck,CIN,PIT,5,2,117.0,18.0,84.6,87.2,-0.45,2.21,"Day Game, on grass",4:10 p.m.,2:30,"43,633"
2,2015-04-06,Joe West,DET,MIN,4,0,109.0,8.0,92.7,96.3,-0.50,0.84,"Day Game, on grass",1:08 p.m.,2:30,"45,030"
3,2015-04-06,Gerry Davis,PHI,BOS,0,8,171.0,18.0,89.5,94.7,-1.00,2.84,"Day Game, on grass",3:08 p.m.,3:01,"45,549"
4,2015-04-06,Ted Barrett,HOU,CLE,2,0,124.0,21.0,83.1,91.9,0.51,1.97,"Night Game, on grass",6:10 p.m.,2:30,"43,753"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13484,2021-10-27,Ron Kulpa,HOU,ATL,7,2,120.0,5.0,95.8,96.7,0.10,0.50,NaN,7:09 p.m.,3:11,"42,833"
13485,2021-10-29,Tom Hallion,ATL,HOU,2,0,146.0,5.0,96.6,93.8,-0.09,0.41,NaN,8:09 p.m.,3:24,"42,898"
13486,2021-10-30,Dan Bellino,ATL,HOU,3,2,153.0,14.0,90.8,94.1,-0.26,1.48,NaN,8:09 p.m.,3:45,"43,125"
13487,2021-10-31,Ted Barrett,ATL,HOU,5,9,171.0,14.0,91.8,88.3,0.61,1.47,NaN,8:15 p.m.,4:00,"43,122"


In [132]:
scrape2_df = ump_scores_df.loc[ump_scores_df.start_time.isnull()]
scrape2_df

,date,umpire,home,away,home_score,away_score,called_pitches,called_wrong,accuracy,consistency,favor,total_run_impact,day_night,start_time,duration,attendance
50,2015-04-10,Bill Miller,LAA,KC,2,4,161.0,19.0,88.2,90.7,-0.20,2.22,NaN,NaN,NaN,NaN
65,2015-04-11,Doug Eddings,LAA,KC,4,6,148.0,15.0,89.9,91.9,0.02,1.60,NaN,NaN,NaN,NaN
70,2015-04-12,Jim Wolf,LAA,KC,2,9,148.0,11.0,92.6,91.2,-0.04,0.84,NaN,NaN,NaN,NaN
169,2015-04-20,Mark Wegner,LAA,OAK,3,6,160.0,26.0,83.8,93.1,-0.07,4.03,NaN,NaN,NaN,NaN
187,2015-04-21,Marty Foster,LAA,OAK,14,1,179.0,12.0,93.3,92.7,0.73,2.95,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13329,2021-09-24,Mark Wegner,LAA,SEA,5,6,154.0,6.0,96.1,95.5,-0.15,0.71,NaN,NaN,NaN,NaN
13331,2021-09-25,Alan Porter,LAA,SEA,14,1,158.0,4.0,97.5,98.7,0.25,0.25,NaN,NaN,NaN,NaN
13346,2021-09-26,Ramon De Jesus,LAA,SEA,1,5,154.0,6.0,96.1,91.6,-0.07,0.53,NaN,NaN,NaN,NaN
13372,2021-09-28,Sean Barber,NYM,MIA,2,1,115.0,10.0,91.3,94.8,0.15,0.81,NaN,NaN,NaN,NaN


In [71]:
##2nd scrape after fixing LAA 3-letter code to ANA
scrape2_df.date = scrape2_df.date.str.replace('-', '')
scrape2_df.loc[scrape2_df.home == 'LAA', 'home'] = 'ANA'
url = 'https://www.baseball-reference.com/boxes/'

scorebox = []
start_time = []
duration = []
day_night = []
for i, j in zip(scrape2_df.date, scrape2_df.home):
    response = requests.get(url + j + '/' + j + i + '0.shtml')
    soup = bs(response.text, 'html.parser')
    print(j)
    print(i)
    print(response)
    scorebox = soup.find_all('div', {'class':'scorebox_meta'})
    start_time.append(scorebox)
    duration.append(scorebox)
    day_night.append(scorebox)
    time.sleep(3)

C:\Users\Alex7\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\Alex7\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


ANA
20150410
<Response [200]>
ANA
20150411
<Response [200]>
ANA
20150412
<Response [200]>
ANA
20150420
<Response [200]>
ANA
20150421
<Response [200]>
ANA
20150422
<Response [200]>
ANA
20150423
<Response [200]>
ANA
20150424
<Response [200]>
ANA
20150426
<Response [200]>
BAL
20150501
<Response [404]>
BAL
20150502
<Response [404]>
BAL
20150503
<Response [404]>
ANA
20150504
<Response [200]>
ANA
20150505
<Response [200]>
COL
20150506
<Response [404]>
ANA
20150506
<Response [200]>
COL
20150506
<Response [404]>
ANA
20150507
<Response [200]>
ANA
20150508
<Response [200]>
CWS
20150509
<Response [404]>
ANA
20150509
<Response [200]>
CWS
20150509
<Response [404]>
ANA
20150510
<Response [200]>
ANA
20150512
<Response [200]>
ANA
20150513
<Response [200]>
COL
20150523
<Response [404]>
COL
20150523
<Response [404]>
ANA
20150525
<Response [200]>
ANA
20150526
<Response [200]>
ANA
20150527
<Response [200]>
BAL
20150528
<Response [404]>
ANA
20150528
<Response [200]>
BAL
20150528
<Response [404]>
ANA
201505

In [76]:
##AFTER EACH SCRAPE RUN CODE TO REVERT CHANGES BEFORE .UPDATING INTO UMP_SCORES_DF
##REMEMBER TO RUN REGEX TO GET VALUES BEFORE UPDATING IN
##AFTER EACH SCRAPE RUN CODE TO REVERT CHANGES BEFORE .UPDATING INTO UMP_SCORES_DF

In [133]:
scrape2_df.loc[scrape2_df.home == 'ANA', 'home'] = 'LAA'

date_list = []
for i in scrape2_df.date:
    date_list.append(i[:4] + '-' + i[4:6] + '-' + i[6:])

scrape2_df.date = date_list

C:\Users\Alex7\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [140]:
start_time_df = pd.DataFrame(start_time)
duration_df = pd.DataFrame(duration)
day_night_df = pd.DataFrame(day_night)

n = 0
if n == 0:
    start_time_df.to_csv('start_time_df2.csv')
    duration_df.to_csv('duration_df2.csv')
    day_night_df.to_csv('day_night_df2.csv')
    n += 1

day_night_df = pd.read_csv('Data\\day_night_df2.csv').drop(columns='Unnamed: 0').set_axis(['day_night'], axis=1).set_index(scrape2_df.index)
start_time_df = pd.read_csv('Data\\start_time_df2.csv').drop(columns='Unnamed: 0').set_axis(['start_time'], axis=1).set_index(scrape2_df.index)
duration_df = pd.read_csv('Data\\duration_df2.csv').drop(columns='Unnamed: 0').set_axis(['duration'], axis=1).set_index(scrape2_df.index)
attendance_df = pd.read_csv('Data\\day_night_df2.csv').drop(columns='Unnamed: 0').set_axis(['attendance'], axis=1).set_index(scrape2_df.index)

day_night_df.day_night = day_night_df.day_night.str.extract(r'((?<=\d\:\d{2}</div><div>).*?(?=</div><div>))')
start_time_df.start_time = start_time_df.start_time.str.extract(r'((?<=Time: ).*?(?= Local))')
duration_df.duration = duration_df.duration.str.extract(r'((?<=n</strong>: ).*?(?=</div>))')
attendance_df.attendance = attendance_df.attendance.str.extract(r'((?<=ce</strong>: ).*?(?=</div>))')

scrape2_df['day_night'] = day_night_df.day_night
scrape2_df['start_time'] = start_time_df.start_time
scrape2_df['duration'] = duration_df.duration
scrape2_df['attendance'] = attendance_df.attendance

ump_scores_df.update(scrape2_df)
ump_scores_df

,date,umpire,home,away,home_score,away_score,called_pitches,called_wrong,accuracy,consistency,favor,total_run_impact,day_night,start_time,duration,attendance
0,2015-04-05,Mike Winters,CHC,STL,0.0,3.0,158.0,17.0,89.2,94.9,1.10,4.36,"Night Game, on grass",7:17 p.m.,3:04,"35,055"
1,2015-04-06,John Hirschbeck,CIN,PIT,5.0,2.0,117.0,18.0,84.6,87.2,-0.45,2.21,"Day Game, on grass",4:10 p.m.,2:30,"43,633"
2,2015-04-06,Joe West,DET,MIN,4.0,0.0,109.0,8.0,92.7,96.3,-0.50,0.84,"Day Game, on grass",1:08 p.m.,2:30,"45,030"
3,2015-04-06,Gerry Davis,PHI,BOS,0.0,8.0,171.0,18.0,89.5,94.7,-1.00,2.84,"Day Game, on grass",3:08 p.m.,3:01,"45,549"
4,2015-04-06,Ted Barrett,HOU,CLE,2.0,0.0,124.0,21.0,83.1,91.9,0.51,1.97,"Night Game, on grass",6:10 p.m.,2:30,"43,753"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13484,2021-10-27,Ron Kulpa,HOU,ATL,7.0,2.0,120.0,5.0,95.8,96.7,0.10,0.50,NaN,7:09 p.m.,3:11,"42,833"
13485,2021-10-29,Tom Hallion,ATL,HOU,2.0,0.0,146.0,5.0,96.6,93.8,-0.09,0.41,NaN,8:09 p.m.,3:24,"42,898"
13486,2021-10-30,Dan Bellino,ATL,HOU,3.0,2.0,153.0,14.0,90.8,94.1,-0.26,1.48,NaN,8:09 p.m.,3:45,"43,125"
13487,2021-10-31,Ted Barrett,ATL,HOU,5.0,9.0,171.0,14.0,91.8,88.3,0.61,1.47,NaN,8:15 p.m.,4:00,"43,122"


In [141]:
scrape3_df = ump_scores_df.loc[ump_scores_df.start_time.isnull()]
scrape3_df

,date,umpire,home,away,home_score,away_score,called_pitches,called_wrong,accuracy,consistency,favor,total_run_impact,day_night,start_time,duration,attendance
305,2015-05-01,Todd Tichenor,BAL,TB,0.0,2.0,112.0,15.0,86.6,98.2,0.25,1.31,NaN,NaN,NaN,NaN
323,2015-05-02,Tim Welke,BAL,TB,4.0,0.0,135.0,17.0,87.4,90.4,-0.11,1.73,NaN,NaN,NaN,NaN
337,2015-05-03,Rob Drake,BAL,TB,4.0,2.0,117.0,11.0,90.6,92.3,0.38,1.30,NaN,NaN,NaN,NaN
373,2015-05-06,Adrian Johnson,COL,ARI,7.0,13.0,202.0,15.0,92.6,93.6,-0.77,2.43,NaN,NaN,NaN,NaN
381,2015-05-06,Bill Miller,COL,ARI,1.0,5.0,129.0,6.0,95.3,96.1,0.36,0.66,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13310,2021-09-23,David Rackley,CLE,CWS,5.0,3.0,120.0,7.0,94.2,95.8,0.11,0.87,NaN,NaN,NaN,NaN
13317,2021-09-24,Doug Eddings,CHC,STL,4.0,12.0,137.0,8.0,94.2,93.4,-0.20,0.84,NaN,NaN,NaN,NaN
13326,2021-09-24,Brian Knight,CHC,STL,5.0,8.0,134.0,14.0,89.6,91.0,-0.02,1.66,NaN,NaN,NaN,NaN
13372,2021-09-28,Sean Barber,NYM,MIA,2.0,1.0,115.0,10.0,91.3,94.8,0.15,0.81,NaN,NaN,NaN,NaN


In [ ]:
covid_2020 = ump_scores_df.date.str.contains('2020')
ump_scores_df = ump_scores_df[~covid_2020].reset_index(drop=True)

filter_nulls = walmart_df.shop_name.isnull()
walmart_df = walmart_df[~filter_nulls].reset_index(drop=True)

In [142]:
ump_scores_df.to_csv('ump_scores_df.csv')